In [1]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse

In [2]:
recipes_clean = pd.read_csv('/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/recipes.csv')
ingredients_keys = pd.read_csv('/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/ingredient_keys.csv')
ingredients_keys.drop(columns='Unnamed: 0',inplace=True)

In [3]:
df_pickle = pd.read_pickle('/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/recipe_list/ingr_map.pkl')
ingredients_clean = df_pickle[['id','replaced','count','raw_ingr']]
# ingredients_clean.head(10)

In [4]:
new_df = recipes_clean[['id','ingredient_ids']].copy()
new_df = pd.DataFrame(new_df)

In [5]:
unique_ingredients_df = ingredients_keys.drop_duplicates().copy()
list_ingredient_ids = unique_ingredients_df['id'].sort_values()

final_df = pd.DataFrame(columns=list_ingredient_ids,index=new_df['id'],data=0)

In [6]:
new_df.head()

,id,ingredient_ids
0,137739,"[7933, 4694, 4795, 3723, 840, 5006, 6270]"
1,31490,"[5481, 6324, 2499, 4717, 6276, 1170]"
2,59389,"[1170, 4918, 6426, 5185, 7099, 5006, 6009, 627..."
3,5289,"[4717, 7474, 2946, 150]"
4,25274,"[2587, 3437, 5002, 3184, 5324, 5068, 5058, 131..."


In [ ]:
%%time
reduced_df = new_df.head(2000)
count =0
for id,recipe in new_df[['id','ingredient_ids']].to_numpy():
    count+=1
    print(f"looking at id: {count}",end="\r")
    columns = ast.literal_eval(recipe)
    final_df.loc[id,columns] = 1

In [ ]:
%%time
save = False
if save == True:
    final_df.to_csv(r'/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/final_df.csv')

In [ ]:
final_sparse = sparse.csr_matrix(final_df.values)

In [ ]:
save = False
if save == True:
    sparse.save_npz(r'/Users/giancarlotissot/code/mariliabreis/ingredient_matching/raw_data/sparse_final_df.npz',final_sparse)

In [ ]:
### TO DO ###
### Use Sparse Matrix

## Input function

In [ ]:
def get_id(ingredient_str):
    # Transforms string input to pre-processed ID
    ingredient_id = ingredients_clean[ingredients_clean['raw_ingr'] ==ingredient_str]
    ingredient_id.reset_index(inplace=True)
    return ingredient_id.loc[0,'id']

def get_name(ingredient_id):
    # Transforms ID back to pre-processed string
    ingredient_name = ingredients_clean[ingredients_clean['id'] == ingredient_id]
    ingredient_name.reset_index(inplace=True)
    return ingredient_name.loc[0,'replaced']

In [ ]:
def find_match(ingredient_id,num_matches):
    # Returns list of ingredient IDs and count of occurances
    total = final_df[final_df[ingredient_id]==1].sum()
    ids = total.sort_values(ascending=False)
    return list(ids.head(num_matches).index)
    
def list_to_names(ingredient_id_list):
    # Takes in list of ingredient IDs and returns list of names
    list_ = []
    for id in ingredient_id_list:
        list_.append(get_name(id))
    return list_

def output_func(input_ingredient,num_matches=10):
    # Combines other functions into a workflow
    num_matches += 1
    result = get_id(input_ingredient)
    ids = find_match(result,num_matches)
    names = list_to_names(ids)
    names.pop(0)
    return names

In [ ]:
# [ingredient_id_1]==1 && [ingredient_id_2]==1

## Processing Order
#get_id -> str to ID
#find_match -> Table
#list_to_names

In [ ]:
output_func('salt')